# Import dependencies


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
PATH_INPUT = '/content/drive/MyDrive/Notebooks/IHLT/Labs/test-gold/STS.input.SMTeuroparl.txt'
PATH_GS = '/content/drive/MyDrive/Notebooks/IHLT/Labs/test-gold/STS.gs.SMTeuroparl.txt'

In [5]:
import nltk
import nltk
from scipy.stats import pearsonr
import pandas as pd
from nltk.metrics import jaccard_distance


In [6]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Load file

In [7]:
dt = pd.read_csv(PATH_INPUT,sep='\t',header=None)
dt = dt.rename(columns={0: 'Set_A', 1: 'Set_B'})
dt['gs'] = pd.read_csv(PATH_GS,sep='\t',header=None)
dt.head()

,Set_A,Set_B,gs
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.50
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.00
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.25
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.50
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.00


# Process data

In [8]:
dt['Set_A'] = dt['Set_A'].apply(lambda x: nltk.sent_tokenize(x))
dt['Set_A'] = dt['Set_A'].apply(lambda x: nltk.word_tokenize(x[0]))

dt['Set_B'] = dt['Set_B'].apply(lambda x: nltk.sent_tokenize(x))
dt['Set_B'] = dt['Set_B'].apply(lambda x: nltk.word_tokenize(x[0]))


In [9]:
print(dt['Set_A'][0])

['The', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.']


In [10]:
dt['Set_A'] = dt['Set_A'].apply(lambda x: nltk.pos_tag(x))
dt['Set_B'] = dt['Set_B'].apply(lambda x: nltk.pos_tag(x))


In [11]:
dt['Set_A'][0]

[('The', 'DT'),
 ('leaders', 'NNS'),
 ('have', 'VBP'),
 ('now', 'RB'),
 ('been', 'VBN'),
 ('given', 'VBN'),
 ('a', 'DT'),
 ('new', 'JJ'),
 ('chance', 'NN'),
 ('and', 'CC'),
 ('let', 'VB'),
 ('us', 'PRP'),
 ('hope', 'VB'),
 ('they', 'PRP'),
 ('seize', 'VB'),
 ('it', 'PRP'),
 ('.', '.')]

In [12]:
wnl = nltk.stem.WordNetLemmatizer()
def lemmatize(p):
    if p[1][0] in {'N','V'}:
        return wnl.lemmatize(p[0].lower(), pos=p[1][0].lower())
    return p[0]

In [13]:
dt['lemma_SetA'] = dt['Set_A'].apply(lambda x: [lemmatize(pair) for pair in x])
dt['lemma_SetB'] = dt['Set_B'].apply(lambda x: [lemmatize(pair) for pair in x])


In [14]:
dt['lemma_SetA'] = dt['lemma_SetA'].apply(set)
dt['lemma_SetB'] = dt['lemma_SetB'].apply(set)

In [15]:
dt['lemma_SetA']

0      {a, now, and, it, ., The, they, let, us, be, g...
1      {certain, amendment, to, ., reference, 7, in, ...
2      {you, include, this, ally, remind, ., that, le...
3      {vote, take, will, ., The, p.m, place, 5.30, a...
4      {,, tired, ., The, fisherman, disappointed, be...
                             ...                        
454    {to, ., the, It, population, integration, our,...
455    {vote, take, will, ., The, p.m, place, 5.30, a...
456    {272, a, to, article, this, ., within, Neither...
457    {you, include, this, ally, remind, ., that, le...
458    {representative, citizen, pontificate, ., abou...
Name: lemma_SetA, Length: 459, dtype: object

In [16]:
dt['jacc_distance'] = ''
for index in dt.index:
  dt['jacc_distance'][index] = 1- jaccard_distance(dt['lemma_SetA'][index], dt['lemma_SetB'][index])
dt.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Set_A,Set_B,gs,lemma_SetA,lemma_SetB,jacc_distance
0,"[(The, DT), (leaders, NNS), (have, VBP), (now,...","[(The, DT), (leaders, NNS), (benefit, VBP), (a...",4.50,"{a, now, and, it, ., The, they, let, us, be, g...","{'s, ', therefore, a, and, seize, it, ., The, ...",0.346154
1,"[(Amendment, NNP), (No, NNP), (7, CD), (propos...","[(Amendment, NNP), (No, NNP), (7, CD), (is, VB...",5.00,"{certain, amendment, to, ., reference, 7, in, ...","{certain, amendment, to, ., reference, 7, in, ...",0.923077
2,"[(Let, VB), (me, PRP), (remind, VB), (you, PRP...","[(I, PRP), (would, MD), (like, VB), (to, TO), ...",4.25,"{you, include, this, ally, remind, ., that, le...","{there, among, would, ,, to, you, this, ally, ...",0.391304
3,"[(The, DT), (vote, NN), (will, MD), (take, VB)...","[(The, DT), (vote, NN), (will, MD), (take, VB)...",4.50,"{vote, take, will, ., The, p.m, place, 5.30, a...","{5.30pm, vote, take, will, The, place, at}",0.545455
4,"[(The, DT), (fishermen, NNS), (are, VBP), (ina...","[(The, DT), (fishermen, NNS), (are, VBP), (ina...",5.00,"{,, tired, ., The, fisherman, disappointed, be...","{,, tired, ., The, fisherman, disappointed, be...",1


In [17]:
pearsonr(dt['gs'], dt['jacc_distance'])[0]

0.4569107458417673

In [18]:
PATH = '/content/drive/MyDrive/Notebooks/IHLT/Labs/lab6_data'
dt['jacc_distance'].to_csv(f"{PATH}/Jaccard_Session3.csv")

**Discussion**
Lemmatization is converts the word to its base form. Additionally, lemmatization takes into account the context and then converts the word to its "_meaningful base form" (Prabhakaran, Selva)_. In comparisson to work tokenization or sentence tokenization, where there we remove the stop words, and punctuations. What can happen is that the last characters of a word with a punctuation mark can be removed (i.e. isn't, it's,we've, etc..) especially where the context can be lost. On top of that, this causes a word to be in its incorrect form, this is an incorrect spelling thus a spelling error which can be propagated forward in the model or project that one is working on if it isn't taken into account. Quick note, that lemmatization would identify the corect base form of specific words: 

"Caring" --> Lemmatization --> "Care"

(Prabhakaran, Selva) [Source](https://www.machinelearningplus.com/nlp/lemmatization-examples-python/)


During the past labs, we've seen the different methods in order to get the jaccard similarity of pairs of sentences. In lemmatization we can get the POS (Part of Speech), which is based off its definition and context. Whilst word tokenization can reduce the "understanding" of the corpus by removing characters from our base word. 

**Conclusion**

In summary, the process of lemmatization is the one that takes into account the context and definition of the word by associating the POS. Taking this into consideration, it is most likely that we use lemmatization in order to analyze different corpus for NLP. 

In order to perform better for any pairs of texts, one would have to understand the meaning of the corpus as a whole, thus lemmatization would be a better choice. In order to make a better analysis of the text we would have to "complete" the workflow: 

Corpus --> Tokenization --> Pre-Processing (Stop-words --> Stemming & Lemmatization) --> Feature Engineering (Bag of Word, TF-IDF, Word Embedding) --> Model Build --> Model Evaluation.

If we follow that path then what we can add is the following: 
- Sentiment Analysis 
- Bag of Words --> Document-Term Matrix (DTM)
- Classification (TF-IDF)

_Sentiment Analysis_ --> This would be added in order to understand how negative or positive a given sentence is, because the words might be similar but what the text is trying to convey can vary depending how the context and definition of the word is used. 

_Bag of Words_ --> is a way to extract features from a document, which converts text into the "matrix of occurence of words in a documents"_(Avinash Navlani_) [Source](https://www.datacamp.com/community/tutorials/text-analytics-beginners-nltk), basically whether given words appeared in a document or not. 

_Classification_ --> Here we'd use TF-IDF, since it measure the amount of information a given word provides accross a document. 

To wrap it all up: 
1. We'd use lemmatization as it contains more information such as context and definition of a word. 
2. If we'd want to improve the analysis of pairs of text, then we could add sentiment analysis, BoW, or classification with a TF-IDF method. From there build a model and then evaluate the model. 